In [88]:
mask = a == b
a[mask] = 0
b[mask] = 0
save('a', a)
save('b', b)

In [85]:
A = 'process/10X_A1_Tile-107.nuclei.tif'
B = 'process_ipynb/10X_A1_Tile-107.nuclei.tif'
a = read(A)
b = read(B)
np.isclose(a, b).all()

False

In [72]:
A = 'process/10X_A1_Tile-107.bases.csv'
B = 'process_ipynb/10X_A1_Tile-107.bases.csv'
# a = read(A)
# b = read(B)
a = pd.read_csv(A)
b = pd.read_csv(B)
# np.isclose(a, b).all()

In [77]:
b

,read,cycle,channel,intensity,cell,i,j,tile,well
0,15,1,A,4099,37,9,966,107,A1
1,15,1,C,1941,37,9,966,107,A1
2,15,1,G,204,37,9,966,107,A1
3,15,1,T,178,37,9,966,107,A1
4,15,2,A,415,37,9,966,107,A1
5,15,2,C,494,37,9,966,107,A1
6,15,2,G,4266,37,9,966,107,A1
7,15,2,T,101,37,9,966,107,A1
8,15,3,A,2953,37,9,966,107,A1
9,15,3,C,1747,37,9,966,107,A1


In [76]:
a

,read,cycle,channel,intensity,cell,i,j,tile,well
0,15,1,A,4099,37,9,966,107,A1
1,15,1,C,1941,37,9,966,107,A1
2,15,1,G,204,37,9,966,107,A1
3,15,1,T,178,37,9,966,107,A1
4,15,2,A,415,37,9,966,107,A1
5,15,2,C,494,37,9,966,107,A1
6,15,2,G,4266,37,9,966,107,A1
7,15,2,T,101,37,9,966,107,A1
8,15,3,A,2953,37,9,966,107,A1
9,15,3,C,1747,37,9,966,107,A1


In [71]:
b.columns

Index([u'read', u'cycle', u'channel', u'intensity', u'cell', u'i', u'j',
       u'tile', u'well'],
      dtype='object')

In [3]:
from ops.imports import *
from ops.process import Align
import ops.firesnake
from ops.firesnake import Snake

# runs example from example_data/ sub-directory of project
home = os.path.dirname(os.path.dirname(ops.__file__))
os.chdir(os.path.join(home, 'example_data'))

df_design = pd.read_csv('design.csv')

THRESHOLD_STD = 300
THRESHOLD_DAPI = 1200
THRESHOLD_CELL = 800
NUCLEUS_AREA = 0.25*150, 0.25*800
WILDCARDS = dict(well='A1', tile='107')

In [45]:
search = 'input/*/10X*{well}_Tile-{tile}.tif'.format(**WILDCARDS)
input_files = natsorted(glob(search))
for f in input_files:
    print(f)

# used to format output filenames
description = parse(input_files[0])
description['subdir'] = 'process_ipynb'
description.pop('cycle');

input/10X_c1-SBS-1/10X_c1-SBS-1_A1_Tile-107.tif
input/10X_c2-SBS-2/10X_c2-SBS-2_A1_Tile-107.tif
input/10X_c3-SBS-3/10X_c3-SBS-3_A1_Tile-107.tif
input/10X_c4-SBS-4/10X_c4-SBS-4_A1_Tile-107.tif
input/10X_c5-SBS-5/10X_c5-SBS-5_A1_Tile-107.tif
input/10X_c6-SBS-6/10X_c6-SBS-6_A1_Tile-107.tif
input/10X_c7-SBS-7/10X_c7-SBS-7_A1_Tile-107.tif
input/10X_c8-SBS-8/10X_c8-SBS-8_A1_Tile-107.tif
input/10X_c9-SBS-9/10X_c9-SBS-9_A1_Tile-107.tif
input/10X_c10-SBS-10/10X_c10-SBS-10_A1_Tile-107.tif
input/10X_c11-SBS-11/10X_c11-SBS-11_A1_Tile-107.tif
input/10X_c12-SBS-12/10X_c12-SBS-12_A1_Tile-107.tif


### load, align, apply Laplacian-of-Gaussian filter (log)

In [32]:
data = np.array([read(f) for f in input_files])
aligned = Snake._align(data, method='DAPI')
save(name(description, tag='aligned'), aligned)

In [33]:
loged = Snake._transform_log(aligned, skip_index=0)
save(name(description, tag='log'), loged)

In [42]:
maxed = Snake._max_filter(loged, 3, remove_index=0)
save(name(description, tag='maxed'), maxed)

### detect candidate reads

In [35]:
std = Snake._compute_std(loged, remove_index=0)
save(name(description, tag='std'), std)

Cast float64 to float32


In [36]:
peaks = Snake._find_peaks(std)
save(name(description, tag='peaks'), peaks)

Cast float64 to float32


### segment nuclei and cells

In [37]:
nuclei = Snake._segment_nuclei(data[0], THRESHOLD_DAPI,
 area_min=NUCLEUS_AREA[0], area_max=NUCLEUS_AREA[1])

save(name(description, tag='nuclei'), nuclei)

In [38]:
cells = Snake._segment_cells(data[0], nuclei, THRESHOLD_CELL)
save(name(description, tag='cells'), cells)

### extract base intensity, call reads, assign to cells

In [64]:
df_bases = Snake._extract_bases(maxed, peaks, cells, 
                        THRESHOLD_STD, wildcards=WILDCARDS)
df_bases.to_csv(name(description, tag='bases', ext='csv'), index=None)

In [65]:
df_reads = Snake._call_reads(df_bases)
filename = name(description, tag='reads', ext='csv')
df_reads.to_csv(filename, index=None)

In [66]:
# maintain same precision as snakemake pipeline
df_reads = pd.read_csv(filename) 
df_cells = Snake._call_cells(df_reads)
df_cells.to_csv(name(description, tag='cells', ext='csv'), index=None)